In [1]:
import asyncio
import time
import cv2
import nats
import numpy as np
from nats.aio.msg import Msg
import nest_asyncio
import random
from asyncio import CancelledError

from project.generated.project.common.proto.Inference_pb2 import InferenceList
from project.common.debug import profiler
from project.common.config_class.profiler import ProfilerConfig
from project.generated.project.common.proto.Image_pb2 import ImageMessage
from project.generated.project.common.proto.Inference_pb2 import Inference

nest_asyncio.apply()

In [2]:
cap = cv2.VideoCapture(0)
nt = await nats.connect("nats://localhost:4222")

image = None
is_done = False


async def on_message(msg: Msg):
    """
    Callback for handling messages received on the `recognition/image_output` topic.
    """
    global image
    global is_done
    try:
        if image is None:
            return

        # Save the image
        cv2.imwrite("right.jpg", image)

        # Write the data to a file
        with open("right_pos.txt", "wb") as file:
            file.write(msg.data)

        is_done = True
    except Exception as e:
        print(f"Error in on_message: {e}")


# Subscribe to the topic
await nt.subscribe("recognition/image_output", cb=on_message)

# Main loop
i = 0
while not is_done:
    i += 1
    if i % 10 != 0:
        await asyncio.sleep(0.1)  # Prevent busy-waiting
        continue

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    image = frame
    _, compressed_image = cv2.imencode(".jpg", frame)

    # Create a unique image ID
    image_id = random.randint(0, 1000000)

    # Prepare and send the message
    msg = ImageMessage(
        image=compressed_image.tobytes(),
        camera_name="camera0",
        is_gray=False,
        id=image_id,
        height=frame.shape[0],
        width=frame.shape[1],
        timestamp=int(time.time() * 1000),
    )

    await nt.publish("recognition/image_input", msg.SerializeToString())
    await nt.flush()

# Release resources
cap.release()
await nt.close()

2024-11-18 14:59:56.830 Python[8130:69524] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
